In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import typing as t
from fdua_competition.vectorstore import build_vectorstore
from fdua_competition.enums import Mode, VectorStoreOption
from fdua_competition.rag import ResearchAssistant, RAG
from fdua_competition.enums import ChatModelOption
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores.base import VectorStore
from pydantic import BaseModel, Field

In [ ]:
# RAGクラスを継承して, 新しいRAGを作成する例

## レスポンスの型式を定義
class FactCheckerResponse(BaseModel):
    score: str = Field(description="スコア")
    reason: str = Field(description="理由")
    reference: list[str] = Field(description="[参照した文章]")


# RAGクラスを継承して新しいRAGクラスを定義
# prompt_template(), build_payload(), output_structure() メソッドをオーバーライドする
class FactChecker(RAG):
    def __init__(
        self, vectorstore: VectorStore, chat_model_option: ChatModelOption = ChatModelOption.AZURE, language: str = "japanese"
    ):
        super().__init__(vectorstore=vectorstore, chat_model_option=chat_model_option)
        self.language = language

    @property
    def prompt_template(self) -> ChatPromptTemplate:
        """プロンプトのテンプレート"""
        return ChatPromptTemplate.from_messages(
            [
                ("system", "情報の正確さを与えられたコンテキストのみ使ってを10点満点で評価して. {language}"),
                ("system", "コンテキスト:\n{context}"),
                ("user", "チェック対象:\n{llm_response}"),
            ]
        )

    def build_payload(self, query: str) -> dict[str, t.Any]:
        """プロンプトテンプレートをこの辞書で埋める"""
        return {"llm_response": query, "language": self.language}

    @property
    def output_structure(self) -> BaseModel:
        """出力型式を定義する"""
        return FactCheckerResponse


# [end: research_assistant]

In [ ]:
vectorstore = build_vectorstore("one_assistant_test", Mode.TEST, VectorStoreOption.CHROMA)

In [ ]:
assistant = ResearchAssistant(vectorstore)

In [ ]:
res = assistant.invoke("電通グループの創業は何年ですか？")
print(res)

In [ ]:
fact_checker = FactChecker(vectorstore)

print("prompt_template:")
print(fact_checker.prompt_template)

query = f"{res.query}: {res.response}"
print(f"\nquery: {query}")

fact_checked_res = fact_checker.invoke(query)
print(f"\nres: {fact_checked_res}")